In [1]:
%pip install psycopg2-binary rich

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datetime import datetime
from collections import defaultdict
from psycopg2 import sql
from rich import print
import psycopg2
import sys

# HELPER FUNCTIONS

In [3]:
def log(message: str) -> None:
    """
    Logs a message to the console.
    """
    print(message)
    with open("sumdb_log.txt", "a") as log_file:
        log_file.write(message + "\n")

In [4]:
def get_column_names(conn, table):
    """
    Fetches and logs the column names of a specified table.
    """
    query = sql.SQL("""
        SELECT column_name 
        FROM information_schema.columns 
        WHERE table_name = %s
        ORDER BY ordinal_position;
    """)
    cur = conn.cursor()
    cur.execute(query, (table,))
    columns = cur.fetchall()
    cur.close()
    return [col[0] for col in columns]

# OVERVIEW OF SUMDB

In [5]:
dirs = '../inputs'
filename = "input.csv"
topicFileName = "topics.txt"

inputFilePath = f"{dirs}/{filename}"
topicFilePath = f"{dirs}/{topicFileName}"

In [6]:
# find len
with open(topicFilePath, 'r') as file:
    true_topics = file.readlines()
    true_topics = [topic.strip() for topic in true_topics]
    
print(f"Number of true topics: {len(true_topics)}")
print(f"True topics: {true_topics}")

Number of true topics: 1

True topics: ['Computer-Science']

In [7]:
db_topic = "logosdb-sumdb"  # using localhost for now
port = "5432"
dbname = "db"  # internal database name
username = "user"
password = "password"

table = "test"  # Name of table to query

In [8]:
# Connect to the database
conn = psycopg2.connect(
    dbname=dbname,
    user=username,
    password=password,
    host=db_topic,
    port=port
)

# Format datetime for readability
formatted_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
log(f"[{formatted_datetime}] Connected to database '{dbname}' on {db_topic}:{port} as '{username}'")

# Get and log column names
column_names = get_column_names(conn, table)
log("Column names in '{}' table: {}".format(table, ", ".join(column_names)))


[2024-08-26 19:20:46] Connected to database 'db' on logosdb-sumdb:5432 as 'user'

Column names in 'test' table: id, chunkstart, chunkend, topic, summary, updatedat

In [9]:
# Prepare and execute the query
query = sql.SQL("SELECT chunkstart, chunkend, topic FROM {}").format(sql.Identifier(table))
with conn.cursor() as cur:
    cur.execute(query)
    rows = cur.fetchall()

# conn.close()

print(rows[:5])

[
    (1, 500, 'Computer-Science'),
    (501, 1000, 'Computer-Science'),
    (1001, 1500, 'Computer-Science'),
    (1501, 2000, 'Computer-Science'),
    (2001, 2500, 'Computer-Science')
]

In [10]:
topic_chunk_count = defaultdict(int)
row_topic_pairs = defaultdict(list)
row_topic_count = defaultdict(int)

for row in rows:
    chunk_start, chunk_end, topic = row

    topic_chunk_count[topic] += 1
    row_topic_pairs[topic].append((chunk_start, chunk_end))
    row_topic_count[topic] += abs(chunk_end - chunk_start) + 1


In [11]:
# log the count of chunks for each topic
log('\nTOPIC NODE ANALYSIS')
for topic, count in topic_chunk_count.items():
    log(f"{topic}: {count} chunks")

log('Chunk Count: ' + str(len(rows)) + ' saved chunks')
log(f'Actual topic node: {len(topic_chunk_count)} nodes')
log(f'Expected topic node: {len(true_topics)} nodes')
log(f'Missing {len(true_topics) - len(topic_chunk_count)} nodes: {set(true_topics) - set(topic_chunk_count.keys())}')


TOPIC NODE ANALYSIS

Computer-Science: 450 chunks

Chunk Count: 450 saved chunks

Actual topic node: 1 nodes

Expected topic node: 1 nodes

Missing 0 nodes: set()

In [12]:

log('\nDATA COUNT ANALYSIS')
row_topic_pairs = dict(sorted(row_topic_pairs.items(), key=lambda x: x[1], reverse=True))

for topic, rows in row_topic_pairs.items():
    log(f"{topic}: {row_topic_count[topic]:,}")
    # log(f"Pairs: {rows}")

total_rows = sum(row_topic_count.values())
log(f'\nTotal rows: {total_rows:,}')


DATA COUNT ANALYSIS

Computer-Science: 224,853

Total rows: 224,853

In [13]:
log('\nCONTINUITY CHECK')
no_gap = True
for topic, chunks in row_topic_pairs.items():
    # Sort the chunks by chunkstart to ensure correct order
    chunks.sort(key=lambda x: x[0])
    for i in range(len(chunks) - 1):
        current_chunk_end = chunks[i][1]
        next_chunk_start = chunks[i + 1][0]
        # Check if there is a gap
        if next_chunk_start != current_chunk_end + 1:
            log(f"Gap found in topic '{topic}' within chunks [{current_chunk_end}, {next_chunk_start}]")
            no_gap = False

if no_gap:
    log('No gap found!')


CONTINUITY CHECK

No gap found!

In [14]:
conn.close()

# Custom Analysis

In [15]:
# this will select the most recent 3 rows
query = sql.SQL("""
    SELECT chunkstart, chunkend, topic, summary 
    FROM {} 
    ORDER BY updatedat DESC
    LIMIT 3;
""").format(sql.Identifier(table))

conn = psycopg2.connect(
    dbname=dbname,
    user=username,
    password=password,
    host=db_topic,
    port=port
)

with conn.cursor() as cur:
    cur.execute(query)
    rows = cur.fetchall()

conn.close()

for i, row in enumerate(rows):
    log(f"CURRENT CHUNKS {i + 1}")
    log(f"Chunk range: ({row[0]}, {row[1]})")
    log(f'Topic: {row[2]}')
    # log(f"Chunk content: {row[3][-1000:]}")

CURRENT CHUNKS 1

Chunk range: (224501, 224853)

Topic: Computer-Science

CURRENT CHUNKS 2

Chunk range: (224001, 224500)

Topic: Computer-Science

CURRENT CHUNKS 3

Chunk range: (223501, 224000)

Topic: Computer-Science

In [16]:
# print 1st 5 rows
first_chunk = rows[0][3]

formatted_chunk = first_chunk.split('\n')

# remove empty lines
formatted_chunk = [line for line in formatted_chunk if line.strip()]

# remove lines with only whitespace or with less than 10 characters
formatted_chunk = [line for line in formatted_chunk if len(line.strip()) > 10]

# print first 5 lines
for i in range(5):
    print(formatted_chunk[i])


224501. datamaker datamaker is a canadian developer and marketer of test data management software datamaker was 
founded by mathieu pelletier in 2020  its products and services aim to provide tools for generating synthetic data 
datamaker is a canadian developer and marketer of test data management software datamaker was founded by mathieu 
pelletier in 2020  its products and services aim to provide tools for generating synthetic data  history  the 
company was started in stealth mode in 2018 the idea was developed inside a big data project the software was 
publically launched in the september 2021 reasons challenges faced the company has receive a preseed funding of 
100k  the startup is part of startup montreal ecosystem   software and services  the software is available in aws 
marketplace as an ec2 ami  there is also a offering on in azure the platform is available as a standalone 
application or in the cloud  references  categorysoftware forcetoc

224502. cornelius schmitt  cornelius schmitt  is a german industrial engineer that has invented the worlds first 
fully 3d printed sneaker he was raised in momberg germany and is a cofounder and currently the ceo of zellerfeld 
shoe company inc  cornelius schmitt  is a german industrial engineer that has invented the worlds first fully 3d 
printed sneaker he was raised in momberg germany and is a cofounder and currently the ceo of zellerfeld shoe 
company inc   education schmitt studied industrial engineering at clausthal university of technology in 
clausthalzellerfeld germany    career during his time studying at clausthal university of technology cornelius 
schmitt was frustrated with the repetitive offering by sneaker companies with the goal of reducing high upfront 
investment costs in footwear production for designers he started developing custom 3d printing machines capable of 
producing shoes at the push of a button   after developing his first 3d printer which schmitt sold to his 
university institute he continued developing and advancing the technology into what now are the best regarded 3d 
printing machines specialized for footwear creation   zellerfeld shoe company keeping the focus on footwear 
cornelius schmitt and lennard stoever cofounded the zellerfeld shoe company inc in 2019 since then zellerfeld has 
gained popularity by collaborating with notable designers such as heron preston kidsuper finn rushtaylor and yoon 
ambush to create 3d printed footwear    after rumours of acquisition offers by kanye west zellerfeld also created 
awareness for their technology when grimes and elon musk ordered a custom shoe for their child x ae axii musk in 
2021 zellerfelds innovative approach of bringing footwear design closer to software development is viewed by many 
industry experts as the future of footwear manufacturing  references

224503. messor security   messor security is the free decentralized and opensource intrusion prevention system with
its own data transfer protocol and hybrid peertopeer network that is capable of performing packet registration and 
realtime analysis and blocking dangerous traffic   it was published by messor limited in 2020   it performs logging
analyzing and searching by content and it is also used to actively block or passively detect a number of attacks 
and probing such as attempts of ddos sql injection bruteforce and other attacks on web apps messor is used to 
prevent intrusion and block attacks   it can work together with other software such as iptables portsentry apache2 
and nginx   messor security is the free decentralized and opensource intrusion prevention system with its own data 
transfer protocol and hybrid peertopeer network that is capable of performing packet registration and realtime 
analysis and blocking dangerous traffic   it was published by messor limited in 2020   it performs logging 
analyzing and searching by content and it is also used to actively block or passively detect a number of attacks 
and probing such as attempts of ddos sql injection bruteforce and other attacks on web apps messor is used to 
prevent intrusion and block attacks   it can work together with other software such as iptables portsentry apache2 
and nginx    key points   ability to protect against scanning regardless of purpose whether exploiting 
vulnerabilities bots search engines or other apps  peertopeer data transmission system   opensource  geopolitical 
independence  what is messor  messor was born as an idea of free independent and decentralized software to ensure 
the security of networks and transparent collection of data on attacks and scans   the main difference between 
messor and other ips is its own network structure everyone who installed messor becomes a part of the messornetwork
that allows himher to obtain daily database updates    database inclusions   a list of ip addresses that the 
network recognized as dangerous which means that attacks were recorded from them repeatedly  lists of ip addresses 
of various bots and crawlers  regular expressions for detecting attacks on useragentgetpostcookies data  regular 
expressions for detecting bots on the useragent  a list of honeypots for scan detection   each participant of the 
messor network must distribute the current database to other participants of the network and send the collected 
data on attacks to the central servers of the network    media about messor   
httpsfullcirclemagazineorgpodcastfullcircleweeklynews245  httpswwwfullcirclemagfrdownload744  
httpswwwopennetruopennewsartshtmlnum56472  download   httpssourceforgenetprojectsmessoropencart  
httpswwwopencartcomindexphproutemarketplaceextensioninfoextensionid42800filtersearchmessor    external links   
httpsmessornetwork  an official site  httpsdocmessornetwork  documentation  httpsgithubcommessornetwork  a 
repository

224504. george geo mueller george geo mueller  19712021 was an inventor and serial entrepreneur his work is widely 
known both for bringing solid state lighting into the mainstream through his companies color kinetics and ecosense 
lighting now korrus and for founding and cocreating robot heart a burning man camp and foundation he was also a 
founder and cocreator of the further future festival in nevada   while muellers early entrepreneurship had proven 
to be highly successful in its own right it is his work with led technology that had a meaningful impact in 
changing the way humanity interacts with light while revolutionary in its scope and reach it was his creation of 
and work with robot heart a collective centered around music that may have had the most resonance as it was as much
about community as it was spectacle hundreds of thousands of revelers have partaken in the ongoing robot heart 
events at burning man and in new york since 2008   born in 1970 in south windsor connecticut and raised in west 
bloomfield michigan a suburb of detroit mueller as a youngster was a natural athlete who played soccer and hockey 
and also excelled in art his passions being sculpture and photography  he also showed early engineering prowess 
while taking apart and reconfiguring such household items as lawn mowers and microwaves a prankster at heart he 
bugged his brothers room at 10 years old and upon acquiring a trs80 computer hacked phone company computers and 
wrote a program to have the computer automate calls to one of their local hang outs custard cone to keep a 
curmudgeonly owner distracted by having him answer the computer generated intermittent prank calls his early 
encounters with leds dated back to high school when he created a kinetic sculpture with robert indianas love 
sculpture visible in the air through peripheral vision i would make pendants by taking a little lr44 watch battery 
hooking it up to an led then gluing them on top of a translucent crystal and attaching a leather cord to form a 
glowing necklace aliens from planet pittsburg how two young roboticists from carnegie mellon pioneered digital 
lighting george geo mueller  19712021 was an inventor and serial entrepreneur his work is widely known both for 
bringing solid state lighting into the mainstream through his companies color kinetics and ecosense lighting now 
korrus and for founding and cocreating robot heart a burning man camp and foundation he was also a founder and 
cocreator of the further future festival in nevada   while muellers early entrepreneurship had proven to be highly 
successful in its own right it is his work with led technology that had a meaningful impact in changing the way 
humanity interacts with light while revolutionary in its scope and reach it was his creation of and work with robot
heart a collective centered around music that may have had the most resonance as it was as much about community as 
it was spectacle hundreds of thousands of revelers have partaken in the ongoing robot heart events at burning man 
and in new york since 2008   born in 1970 in south windsor connecticut and raised in west bloomfield michigan a 
suburb of detroit mueller as a youngster was a natural athlete who played soccer and hockey and also excelled in 
art his passions being sculpture and photography  he also showed early engineering prowess while taking apart and 
reconfiguring such household items as lawn mowers and microwaves a prankster at heart he bugged his brothers room 
at 10 years old and upon acquiring a trs80 computer hacked phone company computers and wrote a program to have the 
computer automate calls to one of their local hang outs custard cone to keep a curmudgeonly owner distracted by 
having him answer the computer generated intermittent prank calls his early encounters with leds dated back to high
school when he created a kinetic sculpture with robert indianas love sculpture visible in the air through 
peripheral vision i would mak

224505. antnio silva footballer antnio joo pereira de albuquerque tavares da silva  born 30 october 2003 is a 
portuguese professional footballer who plays as a centreback for primeira liga club benfica and the portugal 
national team coming through benficas youth system silva impressed during his time with the reserve side winning 
the uefa youth league in 202122 being instrumental part of benficas first european trophy in 60 years he was 
subsequently promoted to the firstteam making his professional debut at age 17 in a breakthrough 202223 season 
silva is a former portugal youth international representing his country at various levels he was chosen in the 
senior squad for the 2022 fifa world cup making shortly after his senior international debut antnio joo pereira de 
albuquerque tavares da silva  born 30 october 2003 is a portuguese professional footballer who plays as a 
centreback for primeira liga club benfica and the portugal national team coming through benficas youth system silva
impressed during his time with the reserve side winning the uefa youth league in 202122 being instrumental part of 
benficas first european trophy in 60 years he was subsequently promoted to the firstteam making his professional 
debut at age 17 in a breakthrough 202223 season silva is a former portugal youth international representing his 
country at various levels he was chosen in the senior squad for the 2022 fifa world cup making shortly after his 
senior international debut club career benfica early career born in viseu silva began his youth career with 
hometown club viseu united aged 11 and also had spells at penalva do castelo and repesenses he quickly stood out 
subsequently began having multiple training sessions with sporting cp while porto also showed an interest in 
signing him but he ended up joining their rivals and his boyhood club benficas youth academy in 2016 struggling 
with homesickness silva returned to his hometown to spend time with his family but with the help of the clubs 
psychologist he managed to overcome his mental health problems and went back to benfica a year later after 
adjusting to life in lisbon 202123 youth league win and firstteam breakthrough on 28 november 2021 silva signed his
first professional contract with benfica b he made his professional debut with benfica b in a 21 ligapro loss to 
mafra on 2 april 2022 during that season silva played a major role in the 202122 uefa youth league where benfica 
u19 topped their group against dynamo kyiv barcelona and bayern munich in the final phases he scored the equaliser 
in a 32 win over midtjylland after his side were two goals behind and in the final he helped his side keep a clean 
sheet in a 60 win over red bull salzburg to help benfica win their first youth league title and their first title 
in european football since the 196162 european cup having appeared for the clubs b team it was expected that silva 
would spend the 202223 season continuing his development there but he began training with the first team after 
impressing newly arrived coach roger schmidt which led him to progress in the teams centrebacks ranks ahead of toms
araujo and jan vertonghen who was away on international duty for belgium following the suspension of nicols 
otamendi he was given his first start by the new coach on 27 august being entrusted in the centre defence alongside
morato and putting a strong performance in a 30 away win over boavista in the primeira liga he subsequently became 
a starter following an injury crisis at the club leading silva to fill the void alongside otamendi who was the only
fit centreback available on 5 september he agreed to a contract extension to 2027 increasing his buyout clause to 
100 million the following day silva made his uefa champions league debut partnering with nicols otamendi in the 
centre of defence keeping a clean sheet in a 20 home win over maccabi haifa  silva was named the leagues defender 
of the month for september after he